<a href="https://colab.research.google.com/github/stormyCobra1199/showcase/blob/main/hdbrtbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
### HDB Resale Transaction bot - by M. W. Ho (c) 2022

!pip install python-telegram-bot
!pip install tabulate

import pandas as pd
import datetime
import logging
import requests
import json
import re
import os
import matplotlib.pyplot as plt
import telegram
from telegram.ext import Updater, InlineQueryHandler, CommandHandler
from tabulate import tabulate
from pandas.plotting import table 
from matplotlib.backends.backend_pdf import PdfPages

PORT = int(os.environ.get('PORT', 8443))
# TOKEN = os.environ["TOKEN"]
# GTOKEN = os.environ["GTOKEN"]
# GID = os.environ["GID"]
from myconf import TOKEN, GTOKEN, GID

def get_url():
    contents = requests.get('https://dog.ceo/api/breeds/image/random').json()
    url = contents['message']
    return url

def get_image_url():
    allowed_extension = ['jpg', 'jpeg', 'png']
    file_extension = ''
    while file_extension not in allowed_extension:
        url = get_url()
        file_extension = re.search("([^.]*)$", url).group(1).lower()
    return url

def bop(update, context):
    url = get_image_url()
    chat_id = update.message.chat.id
    context.bot.send_photo(chat_id=chat_id, photo=url)


###
def start(update, context):
    context.bot.send_message(chat_id=update.effective_chat.id, text="HDBRT Bot - Get latest HDB Resale Transactions.\nExamples: Type '/hs 5 Bedok' to view latest 5Rm sold in Bedok, '/hs French' for French Rd latest sold HDB or '/hs DBSS' for all DBSS flats recently sold etc.")
   
def hs(update, context):
    q=' '.join(context.args)
    if q[0] in ["2","3","4","5"]:
        url = "https://data.gov.sg/api/action/datastore_search?resource_id=f1765b54-a209-4718-8d38-a39237f502b3&fields=month,block,street_name,flat_type,flat_model,resale_price,floor_area_sqm,lease_commence_date,storey_range&sort=month%20desc&limit=20&filters={\"flat_type\":\""+q[0]+" ROOM"+"\",\"town\":\""+q[2:].upper()+"\"}"
        #context.bot.send_message(chat_id=update.effective_chat.id, text=url)
    else:
        url = "https://data.gov.sg/api/action/datastore_search?resource_id=f1765b54-a209-4718-8d38-a39237f502b3&fields=month,block,street_name,flat_type,flat_model,resale_price,floor_area_sqm,lease_commence_date,storey_range&sort=month%20desc&limit=20&q="+q
    response=requests.get(url)
    dataset=response.json()
    df = pd.json_normalize(dataset['result'], "records")  
    df = df.sort_values(by='block').set_index('block').reindex(columns=['street_name','flat_type','flat_model','resale_price','floor_area_sqm','lease_commence_date','storey_range','month'])
    df.rename(columns={'street_name': 'street', 'flat_type': 'type', 'resale_price': 'price', 'lease_commence_date': 'built', 'storey_range': 'floor', 'floor_area_sqm': 'sqm', 'flat_model': 'model', 'month': 'sold'}, inplace=True)
    df.replace(['2 ROOM', '3 ROOM', '4 ROOM', '5 ROOM', 'New Generation', 'Standard', 'EXECUTIVE', 'Maisonette', 'Simplified', 'Improved'],['2','3','4','5','NG','Std','E','M','S','I'], inplace=True) 
    df['type']=df['type']+df['model']
    df.drop('model', axis=1, inplace=True)
    dft=tabulate(df, headers='keys', tablefmt='plain', numalign='left')
    # send text table
    context.bot.send_message(chat_id=update.effective_chat.id, text=dft)
    # send photo
    fig, ax = plt.subplots(figsize=(16,8)) # set size frame
    ax.axis('tight')
    ax.axis('off')
    ax.table(cellText=df.values,colLabels=df.columns,loc='center')
    pp = PdfPages("table.pdf")
    pp.savefig(fig, bbox_inches='tight')
    pp.close()
    context.bot.send_document(chat_id=update.effective_chat.id, document=open('table.pdf', 'rb'))
    ## copy query to gist
    query_url = "https://api.github.com/gists/"+GID+"/comments"
    data = {"body": q}
    headers = {'Authorization': f'token {GTOKEN}'}
    r = requests.post(query_url, headers=headers, data=json.dumps(data))

def errorhand(update, context):
    context.bot.send_message(chat_id=update.effective_chat.id, text="Please enter a valid town or search term. Eg: Type '/hs 5 Bedok' to view latest 5Rm sold in Bedok, '/hs French' for French Rd latest sold HDB or '/hs DBSS' for all DBSS flats recently sold etc.")

###
def main():
    updater = Updater(TOKEN, use_context=True)
    dp = updater.dispatcher
    dp.add_handler(CommandHandler('start', start))
    dp.add_handler(CommandHandler('bop', bop))
    dp.add_handler(CommandHandler('hs', hs))
    dp.add_error_handler(errorhand)
    ##
    # updater.start_webhook(listen="0.0.0.0",
    #                       port=int(PORT),
    #                       url_path=TOKEN)
    # updater.bot.setWebhook('https://doggodemo.herokuapp.com/' + TOKEN)
    #updater.idle()
    updater.start_polling()

if __name__ == '__main__':
    main()


### run in Console F12

function KeepClicking(){
console.log("Clicking");
document.querySelector("colab-connect-button").click() }
setInterval(KeepClicking,60000)

### for keeping alive

import time
t_end = time.time() + 200000 
while time.time() < t_end:
    time.sleep(360)
    print("Stayin' alive...")

### for testing API and response only

### testing return image
import matplotlib.pyplot as plt
ax = plt.subplot(frame_on=False) # no visible frame
ax.xaxis.set_visible(False)  # hide the x axis
ax.yaxis.set_visible(False)  # hide the y axis
table(ax, df)  # where df is your data frame
plt.rcParams["figure.figsize"] = (18, 1)
plt.savefig('mytable.png')